In [ ]:
#############################################################################
#
# Load Required Libraries
#
#####################

# Standard packages
import sys
import csv
import math
import numpy as np
import pandas as pd
from operator import itemgetter
import time
from collections import Counter

# Pipeline, if not using imblearn comment that out and use sklearn's pipeline instead
#from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline, Pipeline

# Some sklearn tools for preprocessing, feature selection and etc
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer, scale, MinMaxScaler
from sklearn.feature_selection import RFE, RFECV, VarianceThreshold, SelectFromModel
from sklearn.feature_selection import SelectKBest, mutual_info_regression, mutual_info_classif, chi2
from sklearn.decomposition import PCA

# Imbalanced Datasets
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss, RandomUnderSampler
from imblearn.metrics import classification_report_imbalanced

# Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import xgboost.sklearn as xgb
from catboost import CatBoostClassifier, cv, Pool
from lightgbm.sklearn import LGBMClassifier

# Model Evaluation
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import make_scorer

# Model Exporting
#from sklearn.externals import joblib
import pickle

# Model Interpretability
import shap
import lime

# pip install tune-sklearn ray[tune]
from tune_sklearn import TuneSearchCV
from tune_sklearn import TuneGridSearchCV

# ML Flow
#import mlflow.sklearn


#Handle annoying warnings
import warnings, sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.ConvergenceWarning)


# %% 
#############################################################################
#
# Global parameters
#
#####################

imb_class=0                                         #Control switch for type of sampling to deal with imbalanced class (0=None, 1=SMOTE, 2=NearMiss, 3=RandomUnderSampler)
cross_val=12                                         #Control Switch for CV
norm_features=1                                     #Normalize features switch
pca = 0
feat_select=0                                       #Control Switch for Feature Selection
fs_type=2                                           #Feature Selection type (1=Stepwise Backwards Removal, 2=Wrapper Select, 3=Univariate Selection)
lv_filter=0                                         #Control switch for low variance filter on features
k_cnt= 20                                           #Number of 'Top k' best ranked features to select, only applies for fs_types 1 and 3
param_tuning = 1                                    #Turn on model parameter tuning
exhaustive_search = 0                               #Turn on if you want exhaustive grid search. Otherwise, it will default to RandomizedSearchCV

#Set global model parameters
rand_st=0                                           #Set Random State variable for randomizing splits on runs

# %%
#############################################################################
#
# Load Data
#
#####################
print('Loading Data...')
start_ts=time.time()

data = pd.read_csv("../../data/clean/model_plays.csv", low_memory = False)
data['target'].value_counts()

# Remove labels
del data['play_id']
del data['game_id']


# Result of Feature Selection - Other Script
data.drop(['play_type_punt', 'Fog', 'ld_outcome_end_of_half', 'pd_expl_pass', 'play_type_field_goal', 'play_type_qb_kneel', 'GameMonth', 'Wind', 
           'ld_opp_outcome_end_of_half', 'ld_opp_outcome_punt', 'ld_outcome_field_goal', 'pd_average_tfl', 'Rain', 'Snow', 'def_fs', 'def_le',
           'def_re', 'def_rolb', 'def_ss', 'fumble', 'home', 'ld_outcome_fumble_lost', 'ld_outcome_interception', 'ld_outcome_touchdown',
           'off_lg', 'off_rg', 'off_te', 'def_cb', 'def_dt', 'def_mlb', 'ld_drive_length', 'ld_expl_pass', 'ld_plays', 'off_rt', 'pd_pass_yard_att',
           'qb_scramble', 'def_lolb', 'ld_opp_outcome_field_goal', 'ld_opp_outcome_interception', 'ld_opp_outcome_no_ld', 'ld_outcome_no_ld',
           'ld_outcome_punt', 'ld_outcome_turnover_on_downs', 'off_c', 'qb_spike', 'play_type_qb_spike' , 'pd_average_plays', 'pd_average_sacks', 'pd_average_top'
           , 'pd_avg_interceptions'], axis = 1, inplace = True)


# Test for run and pass predictors. Comment out if we don't want this
data = data[((data['target'] == 'run') |  (data['target'] == 'pass'))]

# Separate Target from dataset
#firstdata['target'] = data['target'].astype('category')
#target_cat_label = dict(enumerate(data.target.categories))

# Change categorical variables to numerical
data['target'] = data['target'].astype('category').cat.codes
# data['target'] = data['target'].map(target_cat_label)

target = data['target']
del data['target']

# Split prior to any normalization/sampling. Stratify is set understanding that we have imbalanced classes and want to preserve that in the split
sample_train, sample_test, sample_target_train, sample_target_test = train_test_split(data, target, test_size=0.50, stratify = target)

data_train, data_test, target_train, target_test = train_test_split(sample_train, sample_target_train, test_size=0.50, stratify = sample_target_train)


# Calculates Shannon Entropy - a measure of class imbalance 
# As suggested by Simone Romano, PhD. 
# https://stats.stackexchange.com/questions/239973/a-general-measure-of-data-set-imbalance
def balance(seq):
    n = len(seq)
    classes = [(clas,float(count)) for clas,count in Counter(seq).items()]
    k = len(classes)

    H = -sum([ (count/n) * np.log((count/n)) for clas,count in classes]) #shannon entropy
    return H/np.log(k)

print('The balance of classes within the dataset is: ', balance(target_train))
print('If this is far less than 1.0, please consider turning on the imblearn class balancing portions of the pipeline!')

print()    
print("Complete. Data Load Runtime:", time.time()-start_ts)
print()

# %%
#############################################################################
#
# Create pipeline lists 
#
##########################################


print('Creating Model Pipelines...')
start_ts=time.time()

# sklearn
#knn_pipe = []
#dt_pipe = []
#rf_pipe = []
#ab_pipe = []
gb_pipe = []
#svm_pipe = []
#mlp_pipe = []
#xg_pipe = []

skpipes = [gb_pipe]

# %%
#############################################################################
#
# Pre-processing
#
##########################################

print('Normalization Turned On')

# In the future ColumnTransformers can be utilized... actually it will be once this pipeline is finished
'''
preprocessor = ColumnTransformer([("numerical", "passthrough", num_features), 
                                  ("categorical", OneHotEncoder(sparse=False, handle_unknown="ignore"),
                                   cat_features)])
'''

# Normalizing using Standard Scaler
if norm_features == 1:
    for i in range(0, len(skpipes)):
        skpipes[i].append(('scalar'+str(i),StandardScaler()))



     
# Dealing with imbalanced classes
if imb_class == 0:
    pass
elif imb_class == 1:
    # Oversample with SMOTE
    print('Balanced Classes Turned On')
    for i in range(0, len(skpipes)):
        skpipes[i].append(('smote'+str(i), SMOTE(random_state = rand_st)))
        
elif imb_class == 2:
    # Undersample using NearMiss
    print('Balanced Classes Turned On')
    for i in range(0, len(skpipes)):
        skpipes[i].append(('NearMiss'+str(i), NearMiss(version=3)))
    
elif imb_class == 3:
    # Undersample using NearMiss
    print('Balanced Classes Turned On')
    for i in range(0, len(skpipes)):
        skpipes[i].append(('undersample'+str(i), RandomUnderSampler()))
    

# %%
#############################################################################
#
# Feature Selection
#
##########################################

#Low Variance Filter
if lv_filter==1:
    print('Low Variance Filter Turned On', '\n')
    
    for i in range(0, len(skpipes)):
        skpipes[i].append(('variance_threshold'+str(i), VarianceThreshold(threshold = 0.5)))   


#Feature Selection
if feat_select==1:
    print('Feature Selection Turned On', '\n')
    
    ##1) Run Feature Selection #######
    if fs_type==1:
        #Stepwise Recursive Backwards Feature removal
        clf = RandomForestClassifier(n_estimators=200, max_depth=None, min_samples_split=3, criterion='entropy', random_state=None, class_weight="balanced")
        cv_rfc = RFECV(clf, n_features_to_select=k_cnt, step=.1, cv = 5, scoring = 'roc_auc')
            
        for i in range(0, len(skpipes)):
            skpipes[i].append(('rfe_rf'+str(i), cv_rfc))   
        
    if fs_type==2:
        #Wrapper Select via model
        clf = RandomForestClassifier(n_estimators=200, max_depth=None, min_samples_split=3, criterion='entropy', random_state=None)
        sel = SelectFromModel(clf, prefit=False, threshold='mean', max_features=None)                                                           #to select only based on max_features, set to integer value and set threshold=-np.inf
        
        for i in range(0, len(skpipes)):
            skpipes[i].append(('wrapper_rf'+str(i), sel))   

    if fs_type==3:                                                                ######Only work if the Target is binned###########
        #Univariate Feature Selection - Chi-squared
        #will throw error if any negative values in features, so turn off feature normalization, or switch to mutual_info_classif
        print ('Univariate Feature Selection - Chi2: ')
        sel=SelectKBest(chi2, k=k_cnt)
        
        for i in range(0, len(skpipes)):
            skpipes[i].append(('ufs'+str(i), sel)) 


# %%
#############################################################################
#
# Dimensionality Reduction
#
##########################################

# PCA Dimensionality Reduction
if pca == 1:
    print('PCA Turned On', '\n')
    for i in range(0, len(skpipes)):
        skpipes[i].append(('pca'+str(i), PCA(n_components = 0.95)))    
        


# %%
#############################################################################
#
# Add Classifiers
#
##########################################

'''
# Logistic Regression
lr_pipe = Pipeline([('scaler', StandardScaler()),
                     ('classifier', LogisticRegression())
                     ])
'''

# List of Classifiers
classifiers = [#('knn_classifier', KNeighborsClassifier()),
               
               # Tree-based methods
               #('dt_classifier', DecisionTreeClassifier()), 
               #('rf_classifier', RandomForestClassifier()),
               #('ab_classifier', AdaBoostClassifier()),
               
               
               # Support Vector Machines
               #('sv_classifier', SVC()),
               
               # Neural Networks
               #('nn_classifier', MLPClassifier()),
               #('xg_classifier', xgb.XGBClassifier(tree_method = 'gpu_hist')),
               #('xg_classifier', xgb.XGBClassifier()),
               ('gb_classifier', GradientBoostingClassifier())
               #
               ]


for i, c in enumerate(classifiers):
    skpipes[i].append(c) 

# %%
#############################################################################
#
# Create Pipelines
#
##########################################

'''
# Logistic Regression
lr_pipe = Pipeline([('scaler', StandardScaler()),
                     ('classifier', LogisticRegression())
                     ])
'''

# KNN
#knn_pipe = Pipeline(skpipes[0])


# Tree Methods
#dt_pipe = Pipeline(skpipes[0])
#rf_pipe = Pipeline(skpipes[0])
#ab_pipe = Pipeline(skpipes[0])
gb_pipe  = Pipeline(skpipes[0])
#xg_pipe = Pipeline(skpipes[0])

# SVMs
#svm_pipe = Pipeline(skpipes[5])

# Neural Networks
#mlp_pipe = Pipeline(skpipes[4])




# List of all classifier pipelines
pipelines = [gb_pipe]

best_accuracy=0.0
best_classifier=0
best_pipeline=""

# Dictionary of pipelines and classifier types for ease of reference
pipe_dict = {1: 'Decision Tree', 2: 'RandomForest', 3: 'AdaBoost', 4:'GradientBoostedTrees', 5:'svm', 6:'feedforwardnn'}
pipe_list = ['GradientBoostedTrees']

print("Complete. Pipeline Runtime:", time.time()-start_ts)
print()


# %% 
#############################################################################
#
# Hyperparameter Tuning
#
##########################################

best_params = []
best_estimator =[]

  
dt_param = [{'dt_classifier__criterion': ['gini', 'entropy'],
             'dt_classifier__splitter': ['best', 'random'],
             'dt_classifier__max_features': ['auto', 'sqrt', 'log2', None],
             'dt_classifier__max_depth': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21],
             'dt_classifier__min_samples_split': [2, 5, 10, 20, 25, 50, 100],
             'dt_classifier__min_samples_leaf': [1, 2, 4, 8, 16, 32, 64, 128, 256],
             'dt_classifier__class_weight': ['balanced', 'balanced_subsample', None]}]

rf_grid = [{'rf_classifier__n_estimators': [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)],
            'rf_classifier__max_features': ['auto', 'sqrt', 'log2'],
            'rf_classifier__max_depth': [int(x) for x in np.linspace(1, 20, num = 20)],
            'rf_classifier__min_samples_split': [2, 5, 10, 20, 25, 50, 100, 200],
            'rf_classifier__min_samples_leaf': [2, 4, 8, 16, 32, 64, 128],
            'rf_classifier__class_weight':  ['balanced', 'balanced_subsample', None],
            'rf_classifier__bootstrap': [True, False]}]

ab_grid = [{'ab_classifier__n_estimators': [int(x) for x in np.linspace(start = 50, stop = 1000, num = 20)],
           'ab_classifier__learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.5, 1],
           'ab_classifier__algorithm': ['SAMME', 'SAMME.R']}]


gb_grid = [{'gb_classifier__loss': ['deviance', 'exponential'],
           'gb_classifier__learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.5, 1],
           'gb_classifier__n_estimators': [int(x) for x in np.linspace(start = 50, stop = 1000, num = 20)],
           'gb_classifier__max_depth' : [3,4,5,6,7,8,9,10,11,12,13,14,15],
           'gb_classifier__max_features' : ['auto','sqrt','log2',None],
           'gb_classifier__min_samples_split': [2, 5, 10, 20, 25, 50, 100],
           'gb_classifier__min_samples_leaf': [1, 2, 4, 8, 16, 32, 64],
           }]

'''
gb_grid = [{'gb_classifier__loss': ['deviance', 'exponential'],
           'gb_classifier__learning_rate': (0.0001, 1),
           'gb_classifier__n_estimators': (0, 1000),
           'gb_classifier__max_depth' : (3,15),
           'gb_classifier__max_features' : ['auto','sqrt','log2',None],
           'gb_classifier__min_samples_split': (2, 100),
           'gb_classifier__min_samples_leaf': (1, 128)
           }]
'''

nn_grid = [{'nn_classifier__activation': ['identity', 'logistic', 'tanh', 'relu'],
           'nn_classifier__hidden_layer_sizes' : [(100,), (10,), (50,50), (20,20), (100,50), (200,100), (30,30,30)],
           'nn_classifier__solver': ['lbfgs', 'sgd', 'adam'],
           'nn_classifier__alpha' : [0.0001, 0.001, 0.01],
           'nn_classifier__learning_rate' : ['constant', 'invscaling', 'adaptive'],
           'nn_classifier__max_depth' : [3,4,5,6,7,8],
           'nn_classifier__max_features' : ['auto','sqrt','log2',None],
           'nn_classifier__min_samples_split': [2, 5, 10, 20, 25, 50, 100],
           'nn_classifier__min_samples_leaf': [1, 2, 4, 8, 16, 32, 64],
           }]


xg_grid = [{#'xg_classifier__n_estimators': [int(x) for x in np.linspace(start = 1000, stop = 1500, num = 6)],
            'xg_classifier__n_estimators': [1400],
            #'xg_classifier__learning_rate': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.025, 0.05, 0.075, 0.1, 0.125, 0.150, 0.25, 0.5, 1],
            'xg_classifier__learning_rate': [0.01],
            'xg_classifier__min_split_loss': [2],
            #'xg_classifier__min_child_weight': [0, 1, 3, 5, 7],
            'xg_classifier__min_child_weight': [7],
            #'xg_classifier__gamma': [i/10.0 for i in range(0,5)],
            'xg_classifier__gamma': [0],
            #'xg_classifier__max_depth': [3, 5, 7, 9, 11, 13,15, 17, 19, 21, 25],
            'xg_classifier__max_depth': [7],
            #'xg_classifier__booster': ['gbtree', 'gblinear', 'dart'],
            'xg_classifier__booster': ['gbtree'],
            #'xg_classifier__reg_alpha': [0, 0.001, 0.005, 0.1, 0.05, 1e-5, 0.1, 0.25, 0.5, 1, 1.25],
            'xg_classifier__reg_alpha': [0.05],
            'xg_classifier__reg_lambda': [0, 1, 1.1, 1.2, 1.3],
            #'xg_classifier__sampling_method' : ['gradient_based', 'uniform'],
            'xg_classifier__subsample': [0.9],
            #'xg_classifier__subsample': [0.8],
            'xg_classifier__colsample_bytree' : [ 0.6]
            #'xg_classifier__colsample_bytree' : [0.8]
            #'xg_classifier__early_stopping_rounds' : [50],
            #'xg_classifier__eval_metric' : ['auc'],
            #'xg_classifier__eval_set' : [[data_test, target_test]]
           }]


params = [gb_grid]

if param_tuning == 1:
    
    gridhistory = []
    
    #scorers = {'AUC' : 'roc_auc'}
    
    for pipe, grid_param, name in zip(pipelines, params, pipe_list):
        print('Tuning Models...')
        start_ts=time.time()
        if exhaustive_search == 1:
            #gridsearch = GridSearchCV(pipe, grid_param, scoring = 'roc_auc', cv=5, verbose=10, n_jobs=-1) 
            gridsearch = TuneGridSearchCV(pipe, grid_param, scoring = 'roc_auc', cv=5, verbose=10, n_jobs=-1) 
        else:    
            #gridsearch = RandomizedSearchCV(pipe, param_distributions = grid_param, n_iter=5000, scoring = 'roc_auc', cv=5, verbose=20, n_jobs=-1)
            gridsearch = TuneSearchCV(pipe, param_distributions = grid_param, n_iter = 1000, scoring = 'roc_auc', search_optimization='random', verbose=2, n_jobs = 8)
        gridsearch.fit(data_train, target_train)
 
        # Get best results        
        
        print(gridsearch.best_params_, file=open('output.txt', 'a'))
        print(gridsearch.best_score_, file=open('output.txt', 'a'))
        
        # Save optimized model
        gridhistory.append(gridsearch.cv_results_)
        
        filename = name + '.sav'
        best_model = gridsearch.best_estimator_
        pickle.dump(best_model, open(filename, 'wb'))
        best_estimator.append(gridsearch.best_estimator_)
        
        print("Complete. GridSearchCV Runtime:", time.time()-start_ts)
        print()

Loading Data...
The balance of classes within the dataset is:  0.9725161679310111
If this is far less than 1.0, please consider turning on the imblearn class balancing portions of the pipeline!

Complete. Data Load Runtime: 2.293516159057617

Creating Model Pipelines...
Normalization Turned On
Complete. Pipeline Runtime: 0.0

Tuning Models...


Early stopping is not enabled. To enable early stopping, pass in a supported scheduler from Tune and ensure the estimator has `partial_fit`.
Hiding process output by default. To show process output, set verbose=2.
Log sync requires rsync to be installed.


Trial name,status,loc,gb_classifier__learning_rate,gb_classifier__loss,gb_classifier__max_depth,gb_classifier__max_features,gb_classifier__min_samples_leaf,gb_classifier__min_samples_split,gb_classifier__n_estimators
_Trainable_871f64a2,PENDING,,0.001,exponential,15,auto,32,2,900
_Trainable_872027fa,PENDING,,0.01,deviance,3,log2,32,2,300
_Trainable_8720c446,PENDING,,0.1,deviance,7,sqrt,8,20,200
_Trainable_8721608a,PENDING,,0.01,deviance,3,auto,64,10,700
_Trainable_8721fcd2,PENDING,,0.0001,exponential,11,,4,5,300
_Trainable_8722c02e,PENDING,,0.001,exponential,7,sqrt,1,2,950
_Trainable_87235c7e,PENDING,,1,exponential,9,sqrt,64,100,800
_Trainable_8723f8c6,PENDING,,0.01,exponential,12,log2,16,25,300
_Trainable_87249510,PENDING,,0.001,deviance,8,log2,1,20,400
_Trainable_87253158,PENDING,,0.001,deviance,6,,64,25,800


Log sync requires rsync to be installed.
Log sync requires rsync to be installed.
Log sync requires rsync to be installed.
Log sync requires rsync to be installed.
Log sync requires rsync to be installed.
Log sync requires rsync to be installed.
Log sync requires rsync to be installed.


Result for _Trainable_872027fa:
  average_test_score: 0.7130878287542204
  date: 2020-08-06_16-49-24
  done: true
  experiment_id: 06ea6b085133469997a9e5f80b47aa16
  experiment_tag: "3_X_id=ObjectID(ffffffffffffffffffffffff0100008001000000),cv=StratifiedKFold(n_splits=5,\
    \ random_state=None, shuffle=False),early_stopping=False,estimator=Pipeline(steps=[('scalar0',\
    \ StandardScaler()),\n                ('gb_classifier', GradientBoostingClassifier())]),gb_classifier__learning_rate=0.01,gb_classifier__loss=deviance,gb_classifier__max_depth=3,gb_classifier__max_features=log2,gb_classifier__min_samples_leaf=32,gb_classifier__min_samples_split=2,gb_classifier__n_estimators=300,groups=None,max_iters=1,n_jobs=-1,return_train_score=False,scoring=make_scorer(roc_auc_score,\
    \ needs_threshold=True),y_id=ObjectID(ffffffffffffffffffffffff0100008002000000)"
  hostname: DESKTOP-KHTEJHN
  iterations_since_restore: 1
  node_ip: 192.168.0.2
  pid: 21392
  split0_test_score: 0.7116517143007

...

The `experiment_checkpoint` operation took 5.9103310108184814 seconds to complete, which may be a performance bottleneck.
Trial Runner checkpointing failed.
Traceback (most recent call last):
  File "C:\Users\halfc\anaconda3\lib\site-packages\ray\tune\trial_runner.py", line 348, in step
    self.checkpoint()
  File "C:\Users\halfc\anaconda3\lib\site-packages\ray\tune\trial_runner.py", line 279, in checkpoint
    os.rename(tmp_file_name, self.checkpoint_file)
FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\halfc\\ray_results\\_Trainable\\.tmp_checkpoint' -> 'C:\\Users\\halfc\\ray_results\\_Trainable\\experiment_state-2020-08-06_16-48-13.json'


Trial name,status,loc,gb_classifier__learning_rate,gb_classifier__loss,gb_classifier__max_depth,gb_classifier__max_features,gb_classifier__min_samples_leaf,gb_classifier__min_samples_split,gb_classifier__n_estimators,iter,total time (s)
_Trainable_87493626,PENDING,,0.1,exponential,10,sqrt,2,20,500,,
_Trainable_8749f982,PENDING,,0.1,deviance,13,auto,1,2,600,,
_Trainable_874a95cc,PENDING,,0.5,deviance,13,auto,16,5,150,,
_Trainable_874b3218,PENDING,,0.5,deviance,7,,8,20,550,,
_Trainable_874bce5e,PENDING,,0.1,exponential,11,auto,1,10,100,,
_Trainable_874c91be,PENDING,,0.0001,deviance,3,,4,50,1000,,
_Trainable_874d2e02,PENDING,,1,exponential,12,,16,100,900,,
_Trainable_873f98a8,RUNNING,,0.001,exponential,13,auto,32,5,450,,
_Trainable_8741949c,RUNNING,,0.1,exponential,11,,16,50,400,,
_Trainable_8744a202,RUNNING,,0.1,exponential,12,auto,8,50,750,,


Log sync requires rsync to be installed.


Result for _Trainable_874601a8:
  average_test_score: 0.7478614615539791
  date: 2020-08-06_17-49-28
  done: true
  experiment_id: 6f4087ee04664b3e857f739b23522600
  experiment_tag: "62_X_id=ObjectID(ffffffffffffffffffffffff0100008001000000),cv=StratifiedKFold(n_splits=5,\
    \ random_state=None, shuffle=False),early_stopping=False,estimator=Pipeline(steps=[('scalar0',\
    \ StandardScaler()),\n                ('gb_classifier', GradientBoostingClassifier())]),gb_classifier__learning_rate=0.1,gb_classifier__loss=exponential,gb_classifier__max_depth=6,gb_classifier__max_features=auto,gb_classifier__min_samples_leaf=8,gb_classifier__min_samples_split=25,gb_classifier__n_estimators=250,groups=None,max_iters=1,n_jobs=-1,return_train_score=False,scoring=make_scorer(roc_auc_score,\
    \ needs_threshold=True),y_id=ObjectID(ffffffffffffffffffffffff0100008002000000)"
  hostname: DESKTOP-KHTEJHN
  iterations_since_restore: 1
  node_ip: 192.168.0.2
  pid: 21556
  split0_test_score: 0.7453239423

Log sync requires rsync to be installed.
